# Домашно 2

Трениране и подобряване на модели

Предайте същата тетрадка като тази в заданието с нанесените от вас промени. Кръстете файла с факултетния си номер.

# Задача 1

В заданието  имате данните отностно сърдечни заболявания, които са взети от [тук](https://archive.ics.uci.edu/dataset/222/bank+marketing).
Данните, които трябва да прочетете, са файлът `bank.csv`

1.1 Прочетете набора от данни с помощта на `pandas`.

1.2 След което разбийте данните на атрибути и целеви атрибут. в случая целевия атрибут е y.

1.3 Подгответе категорийните променливи, така че да могат да бъдат обработвани като индикатори.

1.4 Разбийте данните на тренировъчно и тестово множество (като 30% от данните да са в тестовото множество). 

Преди да ги разбиете вижте какъв е баланса между двата класа и помислете как трябва да ги разбиете(hint: stratification) 

1.5 Скалирайте данните така, че да имате средно 0 и стандартно отклонение 1.


In [228]:
import numpy as np
import mglearn
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import warnings

from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_moons, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier

warnings.filterwarnings(action='ignore', module='scipy', message='^internal gelsd')
%matplotlib inline

In [229]:
#1.1
heart_disease = pd.read_csv("bank.csv")
#1.2
X = heart_disease.drop(labels=['y'],axis=1)
y = heart_disease['y']
#1.3
X = pd.get_dummies(X)*1
#1.4
pd.Series(y).value_counts(normalize=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=0)
#1.5
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Задача 2 Дървета

Използвайте крос-валидация с 5 гънки(5-fold cross validation), за да обучите и оптимизирате хиперпараметрите за дърво на решенията.

Помислете внимателно как трябва да разбиете гънките при толкова небалансирани класове(hint: stratified k-fold cross-validation).

Помислете как да оценявате модела не забравяйте, че точността не работи в този случай, защото данните са силно небалансирани. Имаме нужда от нещо, което съчетава прецизност и обхват(възвръщаемост).

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `max_depth`
* `min_samples_leaf`
* `max_leaf_nodes`

Вижте как се представя алгоритъма върху тренировачните данни и кои са най-добрите хипер параметри.

In [230]:
# Your code here
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score,make_scorer
#use decision tree
#gridsearch cv = 5 takes care of stratified kfold
# use f1 score 
grid = {
   'max_depth': [2,4,6],
   'min_samples_leaf': [5,10,15],
   'max_leaf_nodes': [10,20,30],
   'random_state':[0]

}
f1_metric =  make_scorer(f1_score,average = 'weighted')
search_tree = GridSearchCV(DecisionTreeClassifier(), param_grid=grid, cv=5,scoring=f1_metric)
search_tree.fit(X_train_scaled,y_train)
# print("All results :\n{}\n",search.cv_results_)
print("Best parameters:\n{}\n".format(search_tree.best_params_))
print("Best cross-validation score: {:.5f}".format(search_tree.best_score_))






Best parameters:
{'max_depth': 6, 'max_leaf_nodes': 30, 'min_samples_leaf': 5, 'random_state': 0}

Best cross-validation score: 0.89118


# Задача 3 Случайна гора

Аналогично на предишната задача но този път със случайна гора

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `n_estimators`
* `max_depth`


In [231]:
# Your code here
from sklearn.ensemble import RandomForestClassifier
grid = {
   'max_depth': [5,20,35],
   'n_estimators': [5,10,15],
   'random_state':[0]

}
f1_metric =  make_scorer(f1_score,average = 'weighted')
search_forest = GridSearchCV(RandomForestClassifier(), param_grid=grid, cv=5,scoring=f1_metric)
search_forest.fit(X_train_scaled,y_train)
print("Best parameters:\n{}\n".format(search_forest.best_params_))
print("Best cross-validation score: {:.5f}".format(search_forest.best_score_))

Best parameters:
{'max_depth': 20, 'n_estimators': 15, 'random_state': 0}

Best cross-validation score: 0.87545


# Задача 4 SVM

Аналогично на предишната задача но този път със svm

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `C`
* `gamma`


In [232]:
# Your code here
from sklearn.svm import SVC
grid = {
   'C': [0.1,1,10],
   'gamma': [0.01,0.1,1],
   'random_state':[0]

}
f1_metric =  make_scorer(f1_score,average = 'weighted')
search_svm = GridSearchCV(SVC(), param_grid=grid, cv=5,scoring=f1_metric)
search_svm.fit(X_train_scaled,y_train)
print("Best parameters:\n{}\n".format(search_svm.best_params_))
print("Best cross-validation score: {:.5f}".format(search_svm.best_score_))

Best parameters:
{'C': 10, 'gamma': 0.01, 'random_state': 0}

Best cross-validation score: 0.88135


# Задача 5 

Повторете задачи 2-4 като изполвате резултатите получени до сега за да избрете нови стойности на същите хипер параметри за които да правите нов grid search.


In [233]:
#Нека разгледаме резултатите от задачи 2 за да придобием по-добра
#представа за тях
pd.DataFrame(search_tree.cv_results_)[['param_max_depth','param_max_leaf_nodes','param_min_samples_leaf','mean_test_score']].sort_values(['mean_test_score'])

,param_max_depth,param_max_leaf_nodes,param_min_samples_leaf,mean_test_score
25,6,30,10,0.878942
0,2,10,5,0.881961
1,2,10,10,0.881961
3,2,20,5,0.881961
4,2,20,10,0.881961
6,2,30,5,0.881961
7,2,30,10,0.881961
16,4,30,10,0.882125
13,4,20,10,0.882125
20,6,10,15,0.882787


In [234]:
# Можем да видим че резултатите са доста близко едни до други, като при комбинация :
# - > по-малко min_samples_leaf + повече max_depth  + повече max_leaf_nodes резултатът леко се подобрява.
# Затова ще пробваме с по 3 параметъра ,които следват горната логика
grid = {
   'max_depth': [8,9,11],
   'min_samples_leaf': [2,3,4],
   'max_leaf_nodes': [35,40,45],
   'random_state':[0]

}
search_tree = GridSearchCV(DecisionTreeClassifier(), param_grid=grid, cv=5,scoring=f1_metric)
search_tree.fit(X_train_scaled,y_train)
# print("All results :\n{}\n",search.cv_results_)
print("Best parameters:\n{}\n".format(search_tree.best_params_))
print("Best cross-validation score: {:.5f}".format(search_tree.best_score_))



Best parameters:
{'max_depth': 8, 'max_leaf_nodes': 45, 'min_samples_leaf': 2, 'random_state': 0}

Best cross-validation score: 0.89268


In [235]:
pd.DataFrame(search_tree.cv_results_)[['param_max_depth','param_max_leaf_nodes','param_min_samples_leaf','mean_test_score']].sort_values(['mean_test_score'])

,param_max_depth,param_max_leaf_nodes,param_min_samples_leaf,mean_test_score
18,11,35,2,0.884320
17,9,45,4,0.884695
21,11,40,2,0.885050
9,9,35,2,0.885686
24,11,45,2,0.886547
12,9,40,2,0.886712
10,9,35,3,0.886755
8,8,45,4,0.887486
5,8,40,4,0.887856
20,11,35,4,0.888200


Леко подобрихме резултатите чрез лека промяна на хиперпараметрите


In [236]:
pd.DataFrame(search_forest.cv_results_)[['param_n_estimators','param_max_depth','mean_test_score']].sort_values(['mean_test_score'])

,param_n_estimators,param_max_depth,mean_test_score
2,15,5,0.847232
1,10,5,0.849716
0,5,5,0.850972
3,5,20,0.871338
7,10,35,0.871555
4,10,20,0.874142
8,15,35,0.874501
6,5,35,0.874748
5,15,20,0.875450


In [237]:
#Тук не можем да забележим толкова голяма корелация между нарастване
# на един параметър и увеличаване на резултатът

In [238]:
# Your code here
grid = {
   'max_depth': [25,40,45],
   'n_estimators': [15,25,35],
   'random_state': [0]

}
f1_metric =  make_scorer(f1_score,average = 'weighted')
search_forest = GridSearchCV(RandomForestClassifier(), param_grid=grid, cv=5,scoring=f1_metric)
search_forest.fit(X_train_scaled,y_train)
print("Best parameters:\n{}\n".format(search_forest.best_params_))
print("Best cross-validation score: {:.5f}".format(search_forest.best_score_))

Best parameters:
{'max_depth': 40, 'n_estimators': 25, 'random_state': 0}

Best cross-validation score: 0.87712


In [239]:
pd.DataFrame(search_forest.cv_results_)[['param_n_estimators','param_max_depth','mean_test_score']].sort_values(['mean_test_score'])

,param_n_estimators,param_max_depth,mean_test_score
5,35,40,0.873829
8,35,45,0.873829
3,15,40,0.874501
6,15,45,0.874501
2,35,25,0.875034
1,25,25,0.875279
0,15,25,0.876718
4,25,40,0.877117
7,25,45,0.877117


In [240]:
#Отново леко подобрение при увеличаване на броя дървета и максималната дълбочина

In [241]:
pd.DataFrame(search_svm.cv_results_)[['param_C','param_gamma','mean_test_score']].sort_values(['mean_test_score'])

,param_C,param_gamma,mean_test_score
0,0.1,0.01,0.830521
1,0.1,0.1,0.830521
2,0.1,1,0.830521
5,1,1,0.830521
8,10,1,0.831592
4,1,0.1,0.833768
7,10,0.1,0.849726
3,1,0.01,0.863899
6,10,0.01,0.881345


In [242]:
#Можем да забележим че прир увеличаване на С и намаляване на gamma,резултатът се увеличава
# Your code here
from sklearn.svm import SVC
grid = {
   'C': [15,25,35],
   'gamma': [0.0001,0.0005,0.01],
   'random_state':[0]

}
f1_metric =  make_scorer(f1_score,average = 'weighted')
search_svm = GridSearchCV(SVC(), param_grid=grid, cv=5,scoring=f1_metric)
search_svm.fit(X_train_scaled,y_train)
print("Best parameters:\n{}\n".format(search_svm.best_params_))
print("Best cross-validation score: {:.5f}".format(search_svm.best_score_))

Best parameters:
{'C': 15, 'gamma': 0.01, 'random_state': 0}

Best cross-validation score: 0.88179


In [243]:
pd.DataFrame(search_svm.cv_results_)[['param_C','param_gamma','mean_test_score']].sort_values(['mean_test_score'])

,param_C,param_gamma,mean_test_score
0,15,0.0001,0.859393
3,25,0.0001,0.862408
6,35,0.0001,0.863841
1,15,0.0005,0.867520
4,25,0.0005,0.870853
7,35,0.0005,0.872004
8,35,0.01,0.878050
5,25,0.01,0.880092
2,15,0.01,0.881794


In [244]:
#Леко подобрение с няколко десетохилядни

# Задача 6

Сравнете как се предстявят всички модели и изберете кой е най-добрия


In [245]:
#На база на 3те резултата,бих казал че най-добрия е класификационното
# дърво,което има най-висок резултат от 3те,а именно 0.892261.Това предполагам
# се дължи на факта,че класификационните дървета са по-прости и често
# могат да overfit-ват,но за сметка на това класификационни дървета са по-бързи